In [17]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
from geopy.distance import great_circle
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor

import time

In [18]:
import sys
sys.path.append(r'../')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [19]:
with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server
name_project

'project_repartrition/output_repar_02-04/'

In [20]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartrition/output_repar_02-04/


In [21]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [22]:
name_project_based = info_basic['name_project_based']
dir_project_based = os.path.join(dir_project_workspace, name_project_based)
print('dir_project_based: ', dir_project_based)

dir_project_based:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-01/


In [23]:
filename = dir_project_based+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic_based = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi_based = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [24]:
key_subworks = info_basic['key_subworks']
key_subworks

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79']

In [25]:
key_subworks_based = info_basic_based['key_subworks']

In [26]:
dir_stack = dir_project+info_basic['dir_stack']
dir_CC = dir_CC_workspace+info_basic_based['name_CC']
#dir_CC = dir_CC_workspace+'CC/CC_40_prewhiten/'

In [27]:
stalistname_all = info_basic_based['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [28]:
ncffile = h5py.File( dir_CC+ 'gather_all.h5','r')
ncfs = ncffile['ncfs'][:]
f = ncffile['f'][:]
count_all = ncffile['count'][:]
ncffile.close()

In [29]:
info_basic_bi['r_max'] = {}

In [30]:
stainfo_all = pd.read_excel(dir_project+info_basic['stalistname'],sheet_name=None)

In [31]:
for key_subwork in key_subworks:
    print("Collecting ",key_subwork,' ...')
    time0 = time.time()

    nf = info_basic['nf']
    #f = info_basic['f']
    stainfo = stainfo_all[key_subwork]
    nsta = len(stainfo.iloc[:,0])
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)
    stalist = stainfo['Station'].tolist()
    lat = stainfo['latitude'].tolist() 
    lon = stainfo['longitude'].tolist()
    ncfs_sum_linear = np.zeros((nPairs,nf),dtype=np.complex64)
    r = np.zeros(nPairs)
    count= np.zeros(nPairs)
    StationPairs = GetStationPairs(nsta)
    names = []
    for i in range(nPairs):
        sta1 = StationPairs[2*i]
        sta2 = StationPairs[2*i+1]
        idx1 = np.min( [int(stalist_all.index(stalist[sta1])),int(stalist_all.index(stalist[sta2]))] )
        idx2 = np.max( [int(stalist_all.index(stalist[sta1])),int(stalist_all.index(stalist[sta2]))] )
        #idx1 = int(stalist_all.index(stalist[sta1]))
        #idx2 = int(stalist_all.index(stalist[sta2]))
        #idx1 = int(stainfo[stainfo[key_pd]==stalist[sta1]].index.values[0])
        #idx2 = int(stainfo[stainfo[key_pd]==stalist[sta2]].index.values[0])
        
        m = 0
        for j in range(nsta_all-idx1,nsta_all):
            m += j
        num = m +idx2 - idx1 -1
        
        ncfs_sum_linear[i,:] = np.nan_to_num(ncfs[num,:])
        count[i] = count_all[num]
        #r[i] = r0[num]
        #if count_all[num] > 0:
        #    ncfs_sum_linear[i,:] = ncfs[num,:]/count_all[num]
        #    count[i] = count_all[num]

        r[i] = great_circle((lat[sta1],lon[sta1]),(lat[sta2],lon[sta2])).km
        #names.append([stalist_all[StationPairs_all[2*num]],stalist_all[StationPairs_all[2*num+1]]])

    outname = str(key_subwork)+'_gather_linear.h5'
    if os.path.exists(dir_stack+outname):
        os.remove(dir_stack+outname)
    ncffile = h5py.File(dir_stack+outname,'w')
    
    ncffile.create_dataset('ncfs',data=ncfs_sum_linear)
    ncffile.create_dataset('r',data=r)
    ncffile.create_dataset('count',data=count)
    ncffile.create_dataset('f',data=f)
    ncffile.create_dataset('StationPairs',data=StationPairs)
    #print(ncffile.keys())
    ncffile.close()
    #np.savez(dir_stack+key_subwork+"_summed-linear.npz",ncfs= ncfs_sum_linear,r = r,stalist=stalist,StationPairs=StationPairs)
    print("Done in ",time.time()-time0," s.")

    info_basic_bi['r_max'][key_subwork] = max(r)

Done in  0.2878611087799072  s.
Done in  1.2771031856536865  s.
Done in  0.975405216217041  s.
Done in  1.06597900390625  s.
Done in  0.6046557426452637  s.
Done in  0.9302632808685303  s.
Done in  0.7629532814025879  s.
Done in  1.2126362323760986  s.
Done in  0.226090669631958  s.
Done in  1.756629467010498  s.
Done in  1.2472758293151855  s.
Done in  1.1878993511199951  s.
Done in  1.479477882385254  s.
Done in  0.24725794792175293  s.
Done in  0.5363574028015137  s.
Done in  1.4685158729553223  s.
Done in  0.7721092700958252  s.
Done in  0.8039407730102539  s.
Done in  0.8322348594665527  s.
Done in  0.9451212882995605  s.
Done in  1.1667063236236572  s.
Done in  0.28367114067077637  s.
Done in  0.6278274059295654  s.
Done in  0.24230742454528809  s.
Done in  0.941107988357544  s.
Done in  0.996903657913208  s.
Done in  0.6480865478515625  s.
Done in  0.5809216499328613  s.
Done in  0.13277053833007812  s.
Done in  1.0648713111877441  s.
Done in  0.2950732707977295  s.
Done in  0.3

In [32]:
np.save(filename_bi,info_basic_bi)